In [1]:
import numpy as np
import pandas as pd
import six
import time
import os
from tqdm import tqdm
import pandas as pd
import bs4

import random
import string

# Load raw scraped dataset

In [2]:
scraped_data = pd.read_pickle('data/scraped/scrape.pickle.gz', compression='gzip')
print(scraped_data.shape)

(130348, 5)


In [11]:
df = scraped_data.reset_index()
df[df['url'].str.contains('Trapiche-Falling-Star-Ch')]

,index,name,image_path,price,url,raw_html
21456,8,Trapiche Falling Star Chardonnay 2000,/product/images/fl_progressive/38447.jpg,7.99,/product/Trapiche-Falling-Star-Chardonnay-2000...,"<div class=""prodItem_wrap"">\n<div class=""prodI..."
21901,3,Trapiche Falling Star Chardonnay 2002,/static/images/nolabel.gif,7.99,/product/Trapiche-Falling-Star-Chardonnay-2002...,"<div class=""prodItem_wrap"">\n<div class=""prodI..."
27805,7,Trapiche Falling Star Chardonnay 2001,/product/images/fl_progressive/52965.jpg,7.99,/product/Trapiche-Falling-Star-Chardonnay-2001...,"<div class=""prodItem_wrap"">\n<div class=""prodI..."
28631,8,Trapiche Falling Star Chardonnay 2003,/product/images/fl_progressive/80073.jpg,7.99,/product/Trapiche-Falling-Star-Chardonnay-2003...,"<div class=""prodItem_wrap"">\n<div class=""prodI..."


In [3]:
scraped_data.head()

,name,image_path,price,url,raw_html
0,Dom Perignon Lenny Kravitz Limited Edition wit...,/product/images/fl_progressive/lmgmud1xsenlouw...,199.97,/product/Dom-Perignon-Lenny-Kravitz-Limited-Ed...,"<div class=""prodItem_wrap"">\n<div class=""prodI..."
1,Louis Roederer Cristal Brut with Two Flutes an...,/product/images/fl_progressive/iqec7e0nf6mzyof...,329.99,/product/Louis-Roederer-Cristal-Brut-with-Two-...,"<div class=""prodItem_wrap"">\n<div class=""prodI..."
2,Laurent-Perrier Cuvee Rose,/product/images/fl_progressive/10521.jpg,79.99,/product/Laurent-Perrier-Cuvee-Rose/10521,"<div class=""prodItem_wrap"">\n<div class=""prodI..."
3,Piper-Heidsieck Cuvee Brut in Travel Case with...,/product/images/fl_progressive/aqi87aqobd3zc56...,79.99,/product/Piper-Heidsieck-Cuvee-Brut-in-Travel-...,"<div class=""prodItem_wrap"">\n<div class=""prodI..."
4,Clarendon Hills Astralis Syrah 2011,/product/images/fl_progressive/qajrqr4d6ttn2pf...,149.99,/product/Clarendon-Hills-Astralis-Syrah-2011/5...,"<div class=""prodItem_wrap"">\n<div class=""prodI..."


# Load scraped descriptions

In [5]:
descriptions = pd.read_csv(
  "data/scraped/descriptions.csv.gz",
  sep='|',
  header=None,
  names=['URL_name', 'desc_raw'],
  encoding='latin1',
  compression='gzip')

descriptions['description'] = descriptions['desc_raw'].apply(
    lambda x: bs4.BeautifulSoup(str(x), 'lxml').get_text()
    )

descriptions.drop('desc_raw', axis=1, inplace=True)

print(descriptions.shape)

(147851, 2)


In [7]:
descriptions.head(3)

,URL_name,description
0,/product/Dom-Perignon-Lenny-Kravitz-Limited-Ed...,Dom Pérignon Vintage 2008 has been distinguish...
1,/product/Louis-Roederer-Cristal-Brut-with-Two-...,Gift pack containing one bottle of 2008 Louis ...
2,/product/Laurent-Perrier-Cuvee-Rose/10521,The Cuvée Rosé from Laurent-Perrier is the mos...


# Clean up descriptions

In [9]:
descriptions['description'] = descriptions['description'].apply(
  lambda x: x.lstrip().rstrip()
)


In [14]:
descriptions.iloc[1,:]

URL_name       /product/Louis-Roederer-Cristal-Brut-with-Two-...
description    Gift pack containing one bottle of 2008 Louis ...
Name: 1, dtype: object

# Merge datasets

In [15]:
scraped_data.drop_duplicates(inplace=True)
descriptions.drop_duplicates(inplace=True)

combined = pd.merge(
    left=scraped_data,
    right=descriptions,
    how='left',
    left_on='url',
    right_on='URL_name'
    )

print(combined.shape)

combined.drop_duplicates(subset='name',inplace=True)
combined.reset_index(drop=True, inplace=True)
print(combined.shape)

(129818, 7)
(125787, 7)


# Save

In [16]:
combined.to_pickle('data/scraped/scraped_with_decs.pickle.gzip', compression='gzip')